Let's try these as opaque pointer models to see if this helps at all.

In [1]:
import os
# os.environ['NUMBA_DUMP_OPTIMIZED'] = '1'

In [8]:
import llvmlite

import numba

# a byte
llvm_type = llvmlite.ir.IntType(64)


class TestStructType(numba.types.Type):
    def __init__(self):
        super().__init__(name="TestStruct")


test_struct_type = TestStructType()


numba.extending.register_model(TestStructType)(numba.datamodel.models.OpaqueModel)


@numba.extending.intrinsic
def create_test_struct(typingctx, int_t):

    sig = test_struct_type(int_t)

    def codegen(context, builder, sig, args):
        a = builder.alloca(llvm_type)
        builder.store(value=args[0], ptr=a)
        return builder.bitcast(a, llvmlite.ir.IntType(8).as_pointer())

    return sig, codegen


@numba.extending.intrinsic
def convert_test_struct(typingctx, test_struct_t):

    sig = numba.types.int64(test_struct_t)

    def codegen(context, builder, sig, args):
        return builder.load(builder.bitcast(args[0], llvmlite.ir.IntType(64).as_pointer()))

    return sig, codegen


@numba.njit
def first_test():
    a = create_test_struct(10)
    # print(convert_test_struct(a))
    return a


@numba.njit
def some_other_thing():
    print(convert_test_struct(first_test()))


some_other_thing()


257


In [6]:
llvmlite.ir.IntType(8).as_pointer()

<<class 'llvmlite.ir.types.PointerType'> i8*>